In [1]:
from langchain_core.tools import tool

@tool
def get_recent_orders_by_status(status, limit=10):
    """
    Retrieve a list of recent orders with the specified status.

    Args:
        status (str): The status of the orders to retrieve.
        limit (int): The maximum number of orders to retrieve (default is 10).

    Returns:
        str: A message confirming the function's execution.
    """
    # 여기서는 실제 데이터베이스에 접근하지 않고, 함수 동작 여부만 확인할 수 있도록
    # 단순한 문자열 메시지를 반환합니다.
    return f"Retrieving recent orders with status '{status}' up to {limit} records."

@tool
def create_order(user, product, quantity):
    """
    Place an order for a product with the specified quantity.

    Args:
        user (User): The user placing the order.
        product (Product): The product to order.
        quantity (int): The quantity of the product to order.

    Returns:
        str: A message confirming that the order has been placed.
    """
    # 여기에 주문을 처리하는 로직을 추가하세요.
    return f"Order placed for {quantity} {product.product_name}(s). Order completed."

@tool
def modify_order(order, new_quantity):
    """
    Modify an existing order to a new quantity.

    Args:
        order (Order): The order to modify.
        new_quantity (int): The new quantity for the order.

    Returns:
        str: A message confirming that the order has been modified.
    """
    # 여기에 주문 변경을 처리하는 로직을 추가하세요.
    return f"Order {order.id} modified to {new_quantity}. Order modification completed."

@tool
def cancel_order(order):
    """
    Cancel an existing order.

    Args:
        order (Order): The order to cancel.

    Returns:
        str: A message confirming that the order has been canceled.
    """
    # 여기에 주문 취소를 처리하는 로직을 추가하세요.
    return f"Order {order.id} canceled. Order cancellation completed."

In [2]:
tools = [get_recent_orders_by_status, create_order, modify_order, cancel_order]
tools

[StructuredTool(name='get_recent_orders_by_status', description="get_recent_orders_by_status(status, limit=10) - Retrieve a list of recent orders with the specified status.\n\nArgs:\n    status (str): The status of the orders to retrieve.\n    limit (int): The maximum number of orders to retrieve (default is 10).\n\nReturns:\n    str: A message confirming the function's execution.", args_schema=<class 'pydantic.v1.main.get_recent_orders_by_statusSchema'>, func=<function get_recent_orders_by_status at 0x0000023DD4EFAC20>),
 StructuredTool(name='create_order', description='create_order(user, product, quantity) - Place an order for a product with the specified quantity.\n\nArgs:\n    user (User): The user placing the order.\n    product (Product): The product to order.\n    quantity (int): The quantity of the product to order.\n\nReturns:\n    str: A message confirming that the order has been placed.', args_schema=<class 'pydantic.v1.main.create_orderSchema'>, func=<function create_order 

In [10]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)
model_with_tools = model.bind_tools(tools)

In [17]:
from langgraph.graph import END, MessageGraph

builder = MessageGraph()
builder.add_node("handle_order", model_with_tools)

In [18]:
from langgraph.prebuilt import ToolNode

tool_node = ToolNode(tools)
builder.add_node("order_tools", tool_node)

In [19]:
builder.add_edge("order_tools", END)

In [20]:
builder.set_entry_point("handle_order")

In [21]:
from typing import Literal, List
from langchain_core.messages import BaseMessage

def router(state: List[BaseMessage]) -> Literal["order_tools", "__end__"]:
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "order_tools"
    else:
        return "__end__"

builder.add_conditional_edges("handle_order", router)

In [22]:
runnable = builder.compile()
runnable

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden']}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<__root__>(recurse=True, writes=[ChannelWriteEntry(channel='__root__', value=<object object at 0x0000023DD511B2F0>, skip_none=True, mapper=None)]), ChannelWrite<start:handle_order>(recurse=True, writes=[ChannelWriteEntry(channel='start:handle_order', value='__start__', skip_none=False, mapper=None)])]), 'handle_order': PregelNode(config={'tags': []}, channels=['__root__'], triggers=['start:handle_order'], writers=[ChannelWrite<handle_order,__root__>(recurse=True, writes=[ChannelWriteEntry(channel='handle_order', value='handle_order', skip_none=False, mapper=None), ChannelWriteEntry(channel='__root__', value=<object object at 0x0000023DD511B2F0>, skip_none=True, mapper=None)]), _route(recurse=True, _is_channel_writer=True)]), 'order_tools': PregelNode(config={'tags': []}, channels=['__root__'], triggers=['branch:handle_order:router

In [23]:
from langchain_core.messages import HumanMessage

response = runnable.invoke(HumanMessage("나는 '나들'이라고 해"))
response


[HumanMessage(content="나는 '나들'이라고 해", id='03c55723-7ed5-42bd-a2b4-006031d4e002'),
 AIMessage(content="안녕하세요, '나들'님! 무엇을 도와드릴까요?", response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 370, 'total_tokens': 399}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-982c5c36-af90-40a5-a102-db3351191a12-0')]

In [24]:
response = runnable.invoke(HumanMessage("주문 상태가 '주문 완료'인 주문 내역을 알고 싶어"))
response

[HumanMessage(content="주문 상태가 '주문 완료'인 주문 내역을 알고 싶어", id='01c7646b-28fd-4be2-834d-2054c5f8cabd'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bvYrftFX4XlKWwscEihjSYBC', 'function': {'arguments': '{"status":"주문 완료"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 385, 'total_tokens': 406}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-9efb900c-40c8-49c0-bcaf-9f26400f413f-0', tool_calls=[{'name': 'get_recent_orders_by_status', 'args': {'status': '주문 완료'}, 'id': 'call_bvYrftFX4XlKWwscEihjSYBC'}]),
 ToolMessage(content="Retrieving recent orders with status '주문 완료' up to 10 records.", name='get_recent_orders_by_status', id='235a5f63-3021-4029-887e-7161a370f2bd', tool_call_id='call_bvYrftFX4XlKWwscEihjSYBC')]

In [26]:
response = runnable.invoke(HumanMessage("떡케익 3개 주문할래요"))
response

[HumanMessage(content='떡케익 3개 주문할래요', id='74dc2e26-3eaf-4d8c-b8cb-501d01298702'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_lT8RDxYsHYgkFxS8grRjaOFe', 'function': {'arguments': '{"status":"pending"}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 376, 'total_tokens': 393}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-aebd3c10-1052-4029-907e-d0939d7aef7a-0', tool_calls=[{'name': 'get_recent_orders_by_status', 'args': {'status': 'pending'}, 'id': 'call_lT8RDxYsHYgkFxS8grRjaOFe'}]),
 ToolMessage(content="Retrieving recent orders with status 'pending' up to 10 records.", name='get_recent_orders_by_status', id='a9236dcf-bc08-462c-9e95-05575f20911c', tool_call_id='call_lT8RDxYsHYgkFxS8grRjaOFe')]

In [27]:
response = runnable.invoke(HumanMessage("주문을 변경하고 싶어"))
response

[HumanMessage(content='주문을 변경하고 싶어', id='1fd1dca2-d949-4608-97a4-fdbbac5d9b23'),
 AIMessage(content='주문을 변경하고 싶으신가요? 주문을 변경하려면 어떤 주문을 수정하고 싶으신지 알려주세요. 주문 번호나 다른 식별자를 제공해주시면 주문을 수정할 수 있습니다.', additional_kwargs={'tool_calls': [{'id': 'call_YfD7iKrTDvZoXA0OxenkA1Br', 'function': {'arguments': '{"status":"pending","limit":5}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 369, 'total_tokens': 462}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-b56aadf6-fccb-4b1b-afbb-dce1a5d21c5c-0', tool_calls=[{'name': 'get_recent_orders_by_status', 'args': {'status': 'pending', 'limit': 5}, 'id': 'call_YfD7iKrTDvZoXA0OxenkA1Br'}]),
 ToolMessage(content="Retrieving recent orders with status 'pending' up to 5 records.", name='get_recent_orders_by_status', id='287fe229-2929-48ce-8727-c12bd3a26e0d', tool_call_id='call_YfD7iKrTDvZoXA0OxenkA1Br')]

In [29]:
response = runnable.invoke(HumanMessage("주문 번호3의 주문을 취소하고 싶어"))
response

[HumanMessage(content='주문 번호3의 주문을 취소하고 싶어', id='92168dd9-cd46-4f36-91f0-69a926299683'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_8GSFtAwoHNwH7C2tEw49KKSR', 'function': {'arguments': '{"status":"pending","limit":1}', 'name': 'get_recent_orders_by_status'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 377, 'total_tokens': 398}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2bfa0ccb-6813-4d68-bde9-8dae228ef3d9-0', tool_calls=[{'name': 'get_recent_orders_by_status', 'args': {'status': 'pending', 'limit': 1}, 'id': 'call_8GSFtAwoHNwH7C2tEw49KKSR'}]),
 ToolMessage(content="Retrieving recent orders with status 'pending' up to 1 records.", name='get_recent_orders_by_status', id='3b9e8fe1-74e8-4245-bea3-0bcf1750f812', tool_call_id='call_8GSFtAwoHNwH7C2tEw49KKSR')]